# Title
[]()

In [2]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")

In [3]:
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
# from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict, create_text_dict_from_folder
from response_processing import revive_chatbot, revive_chatbot_dict, save_instance_to_dict
from wrangling import filter_df_all_conditions

In [83]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

# set up

In [4]:
  references_df_dict = dict()

# Update search_article function

## Iteration 1

In [9]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 
from sources import retrieve_citation, extract_pubmed_details, pubmed_details_by_title, add_pubmed_details, compare_columns, create_sources_table

def search_article(title, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'term': title_without_not,
        'field': 'title',
        'retmax': 5,
        'retmode': 'json'
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(f'[{string.punctuation}]', '', title).lower().strip()

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            result = retrieve_citation(id_list[0], api_key).decode('utf-8')
            cleaned_result = re.sub(f'[{string.punctuation}]', '', result).lower().strip()
            for article_id in id_list:
                result = retrieve_citation(article_id, api_key).decode('utf-8')
                if cleaned_title in cleaned_result:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                    return result
            print('Article title not found in PMIDs.')
            print(f'\tInput title: {title.lower().strip()}')
            # print(f'Result title: {re.sub(r":", r"", result.lower())}')
            return id_list        
    except:
        print('Article not found.')
        return id_list 

iteration_id = 1
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Check these PMIDs: ['37524415', '37512076', '37512059', '37494560', '37490247']
	Input title: sleep and obesity
	Result title: poor sleep quality and its associated neurocognitive function in children with obesity with or without obstructive sleep apnea
	Cleaned input title: sleep and obesity
	Cleaned result title: poor sleep quality and its associated neurocognitive function in children with obesity with or without obstructive sleep apnea

Flagged: 
	Article title: Sleep and obesity
	PubMed title: Poor sleep quality and its associated neurocognitive function in children with obesity with or without obstructive sleep apnea.



,id,title,body,sent_to_sources,section,publication,pubmed_title,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text,flag_title
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,Poor sleep quality and its associated neurocog...,"To evaluate the associations of OSA severity, ...",Sleep &amp; breathing = Schlaf &amp; Atmung,"Chun Ting Au, Giorge Voutsas, Sherri Lynne Kat...",2023,,,,,,10.1007/s11325-023-02886-1,Sleep and Obesity\r\n\r\nObstructive sleep apn...,True


## iteration 2 [refresh kernel]

In [9]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': title_without_not,
        # 'field': 'title',
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract = re.search(r'<AbstractText.*?>(.*?)</AbstractText>', record_string)
    abstract = abstract.group(1) if abstract else ''

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 2
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Check these PMIDs: ['37543916', '37543699', '37543151', '37539611', '37538458']
	Input title: sleep and obesity
	Result title: retracted clinical evaluation of the mediumterm efficacy of laparoscopic sleeve gastrectomy against obstructive sleep apneahypopnea syndrome in obese patients
	Cleaned input title: sleep and obesity
	Cleaned result title: retracted clinical evaluation of the mediumterm efficacy of laparoscopic sleeve gastrectomy against obstructive sleep apneahypopnea syndrome in obese patients

Flagged: 
	Article title: Sleep and obesity
	PubMed title: Retracted: Clinical Evaluation of the Medium-Term Efficacy of Laparoscopic Sleeve Gastrectomy against Obstructive Sleep Apnea-Hypopnea Syndrome in Obese Patients.



,id,title,body,sent_to_sources,section,publication,pubmed_title,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text,flag_title
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,Retracted: Clinical Evaluation of the Medium-T...,[This retracts the article DOI: 10.1155/2022/7...,Computational and mathematical methods in medi...,Computational And Mathematical Methods In Medi...,2023,,2023,,9848635,,10.1155/2023/9848635,Sleep and Obesity\r\n\r\nObstructive sleep apn...,True


## 2.1

In [11]:
def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': title_without_not,
        # 'field': 'title',
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 

iteration_id = 2.1
title = 'Sleep and obesity'
publication = 'Curr Opin Clin Nutr Metab Care'
references_df_dict[iteration_id] = search_article(title, publication, api_key)

Search term: (Sleep and obesity [ti])
Check these PMIDs: ['37543916', '37543699', '37543151', '37539611', '37538458']
	Input title: sleep and obesity
	Result title: retracted clinical evaluation of the mediumterm efficacy of laparoscopic sleeve gastrectomy against obstructive sleep apneahypopnea syndrome in obese patients
	Cleaned input title: sleep and obesity
	Cleaned result title: retracted clinical evaluation of the mediumterm efficacy of laparoscopic sleeve gastrectomy against obstructive sleep apneahypopnea syndrome in obese patients



## 2.2

In [13]:
def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': title_without_not,
        # 'field': 'title',
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 

iteration_id = 2.2
title = 'Sleep and obesity'
publication = 'Curr Opin Clin Nutr Metab Care'
references_df_dict[iteration_id] = search_article(title, publication, api_key)

Search term: (Curr Opin Clin Nutr Metab Care [ta])
Check these PMIDs: ['37543916', '37543699', '37543151', '37539611', '37538458']
	Input title: sleep and obesity
	Result title: retracted clinical evaluation of the mediumterm efficacy of laparoscopic sleeve gastrectomy against obstructive sleep apneahypopnea syndrome in obese patients
	Cleaned input title: sleep and obesity
	Cleaned result title: retracted clinical evaluation of the mediumterm efficacy of laparoscopic sleeve gastrectomy against obstructive sleep apneahypopnea syndrome in obese patients



## 2.3

In [17]:
def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not}[ti]) AND ({publication}[ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': title_without_not,
        # 'field': 'title',
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 

iteration_id = 2.2
# title = 'Sleep+and+obesity'
# publication = 'Curr+Opin+Clin+Nutr+Metab+Care'
title = 'Sleep and obesity'
publication = 'Curr Opin Clin Nutr Metab Care'
references_df_dict[iteration_id] = search_article(title, publication, api_key)

Search term: (Sleep+and+obesity[ti]) AND (Curr+Opin+Clin+Nutr+Metab+Care[ta])


## Iteration 3

In [18]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        # 'field': 'title',
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract = re.search(r'<AbstractText.*?>(.*?)</AbstractText>', record_string)
    abstract = abstract.group(1) if abstract else ''

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 3
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])


,id,title,body,sent_to_sources,section,publication,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,This review summarizes the most recent evidenc...,Current opinion in clinical nutrition and meta...,"Guglielmo Beccuti, Silvana Pannain",2011,,14,4,402,412,10.1097/MCO.0b013e3283479109,Sleep and Obesity\r\n\r\nObstructive sleep apn...


In [20]:
references_df_dict[iteration_id]['abstract'].values

array(['This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.'],
      dtype=object)

In [22]:
pmid = '21659802'
retrieve_citation(pmid, api_key)

b'<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2023//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_230101.dtd">\n<PubmedArticleSet>\n<PubmedArticle><MedlineCitation Status="MEDLINE" Owner="NLM"><PMID Version="1">21659802</PMID><DateCompleted><Year>2011</Year><Month>10</Month><Day>06</Day></DateCompleted><DateRevised><Year>2022</Year><Month>04</Month><Day>08</Day></DateRevised><Article PubModel="Print"><Journal><ISSN IssnType="Electronic">1473-6519</ISSN><JournalIssue CitedMedium="Internet"><Volume>14</Volume><Issue>4</Issue><PubDate><Year>2011</Year><Month>Jul</Month></PubDate></JournalIssue><Title>Current opinion in clinical nutrition and metabolic care</Title><ISOAbbreviation>Curr Opin Clin Nutr Metab Care</ISOAbbreviation></Journal><ArticleTitle>Sleep and obesity.</ArticleTitle><Pagination><StartPage>402</StartPage><EndPage>412</EndPage><MedlinePgn>402-12</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10.109

## Iteration 4

In [25]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract = re.search(r'<AbstractText.*?>(.*?)</AbstractText>', record_string)
    abstract = ''.join([group for group in abstract.group()])
    # abstract = ''.join([group for group in abstract.group() if group != None else ''])

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])


array(['<AbstractText Label="PURPOSE OF REVIEW" NlmCategory="OBJECTIVE">This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.</AbstractText>'],
      dtype=object)

## 4.1

In [26]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'<AbstractText.*?>(.*?)</AbstractText>', record_string)
    abstract = ''.join([group for group in abstract_matches])
    # abstract = ''.join([group for group in abstract.group() if group != None else ''])

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])


array(['This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.Sleep is an important modulator of neuroendocrine function and glucose metabolism and sleep loss has been shown to result in metabolic and endocrine alterations, including decreased glucose tolerance, decreased insulin sensitivity, increased evening concentrations of

In [27]:
references_df_dict[iteration_id]

,id,title,body,sent_to_sources,section,publication,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,This review summarizes the most recent evidenc...,Current opinion in clinical nutrition and meta...,"Guglielmo Beccuti, Silvana Pannain",2011,,14,4,402,412,10.1097/MCO.0b013e3283479109,Sleep and Obesity\r\n\r\nObstructive sleep apn...


## 4.2 Include abstract section titles

In [28]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)

    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    abstract = ''.join([group for group in abstract_matches])
    # abstract = ''.join([group for group in abstract.group() if group != None else ''])

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4.2
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])


array(['<AbstractText Label="PURPOSE OF REVIEW" NlmCategory="OBJECTIVE">This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.</AbstractText><AbstractText Label="RECENT FINDINGS" NlmCategory="RESULTS">Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.</AbstractText><AbstractText Label="SUMMARY" NlmCategory="CONCLUSIONS">Sleep is an important modulator of neuroendocrine functio

## 4.21

In [53]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    # print(f'Data keys: {data.keys()}')
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    cleaned_abstract_sections = []
    for match in abstract_matches:
        # clean_match = re.sub(r'<AbstractText.*?Label="(.?)".*?>(.*?)</AbstractText>', r'\1: \2', match)
        clean_match = re.sub(r'<AbstractText.*?(Label=".*".*?>.*?)</AbstractText>', r'\1', match)
        cleaned_abstract_sections.append(clean_match)
        
    abstract = ''.join([group for group in cleaned_abstract_sections])
    # abstract = 'abstract'

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4.2
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])


array(['Label="PURPOSE OF REVIEW" NlmCategory="OBJECTIVE">This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.Label="RECENT FINDINGS" NlmCategory="RESULTS">Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.Label="SUMMARY" NlmCategory="CONCLUSIONS">Sleep is an important modulator of neuroendocrine function and glucose metabolism and sleep loss has been shown to result in meta

In [52]:
references_df_dict[iteration_id]['abstract'].values

array(['abstract'], dtype=object)

# 4.22

In [63]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    # print(f'Data keys: {data.keys()}')
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    print(f'Number of abstract sections: {len(abstract_matches)}')
    cleaned_abstract_sections = []
    for match in abstract_matches:
        clean_match = re.sub(r'<AbstractText.*?((?:Label=".*").*?>.*)</AbstractText>', r'\1', match)
        clean_match = re.sub(r'Label="(.*?)".*?>(.*)', r'\1: \2', clean_match)
        cleaned_abstract_sections.append(clean_match)
        
    abstract = ''.join([f'{group}\n' for group in cleaned_abstract_sections])
    # abstract = 'abstract'

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4.22
article_limit = 1

text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3


array(['PURPOSE OF REVIEW: This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.\nRECENT FINDINGS: Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.\nSUMMARY: Sleep is an important modulator of neuroendocrine function and glucose metabolism and sleep loss has been shown to result in metabolic and endocrine alterations, including decreased glucose tolerance, decreased insulin

In [64]:
text_df = get_table(table='gpt_queue', limit=article_limit, order='ASC') # db_orm.py
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 1
Search term: Mediterranean diet adherence is associated with lower dementia risk, independent of genetic predisposition: findings from the UK Biobank prospective cohort study [ti]
Number of abstract sections: 4


array(['BACKGROUND: The identification of effective dementia prevention strategies is a major public health priority, due to the enormous and growing societal cost of this condition. Consumption of a Mediterranean diet (MedDiet) has been proposed to reduce dementia risk. However, current evidence is inconclusive and is typically derived from small cohorts with limited dementia cases. Additionally, few studies have explored the interaction between diet and genetic risk of dementia.\nMETHODS: We used Cox proportional hazard regression models to explore the associations between MedDiet adherence, defined using two different scores (Mediterranean Diet Adherence Screener [MEDAS] continuous and Mediterranean diet Pyramid [PYRAMID] scores), and incident all-cause dementia risk in 60,298 participants from UK Biobank, followed for an average 9.1&#xa0;years. The interaction between diet and polygenic risk for dementia was also tested.\nRESULTS: Higher MedDiet adherence was associated with lower 

In [67]:
iteration_id = 4.221
text_df= pd.DataFrame({
    'title': 'Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output',
    'publication': 'Appl Physiol Nutr Metab',
    'body': 'body'
}, index=[0])
# text_df

references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Search term: (Calcaneal tendon stiffness is  associated with dynamic time-dependent contractile output [ti]) AND (Appl Physiol Nutr Metab [ta])
Number of abstract sections: 1


array(["<AbstractText>The ability to rapidly generate muscular torque and velocity is important in specialized activities and daily tasks of living. Tendon stiffness is one factor in the neuromuscular system that influences musculoskeletal torque transmission. Previous studies have reported weak-to-moderate correlations between tendon stiffness and rate of torque development (RTD). However, these correlations have been reported only for isometric contractions, which may not be relevant to contractions involving joint rotation (i.e., dynamic). The purpose was to investigate the effect of calcaneal tendon stiffness on the dynamic RTD and rate of velocity development (RVD) in plantar flexor muscles. Young adult males (<i>n</i>&#xa0;=&#xa0;13) and females (<i>n</i>&#xa0;=&#xa0;2) performed prone isometric- and isotonic-mode maximal voluntary plantar flexion contractions (MVC). Ultrasound imaging was used to quantify tendon morphological characteristics to estimate Young's elastic modulus (

## 4.3

In [74]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    # print(f'Data keys: {data.keys()}')
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    print(f'Number of abstract sections: {len(abstract_matches)}')
    if len(abstract_matches) > 1:
        cleaned_abstract_sections = []
        for match in abstract_matches:
            # clean_match = re.sub(r'<AbstractText(.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
            cleaned_abstract_sections.append(clean_match)
            
        abstract = ''.join([f'{group}\n' for group in cleaned_abstract_sections])
    else:
        abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''

    # abstract = 'abstract'

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4.22
article_limit = 1

# text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
text_df= pd.DataFrame({
    'title': 'Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output',
    'publication': 'Appl Physiol Nutr Metab',
    'body': 'body'
}, index=[0])
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Search term: (Calcaneal tendon stiffness is  associated with dynamic time-dependent contractile output [ti]) AND (Appl Physiol Nutr Metab [ta])
Number of abstract sections: 1


array(["The ability to rapidly generate muscular torque and velocity is important in specialized activities and daily tasks of living. Tendon stiffness is one factor in the neuromuscular system that influences musculoskeletal torque transmission. Previous studies have reported weak-to-moderate correlations between tendon stiffness and rate of torque development (RTD). However, these correlations have been reported only for isometric contractions, which may not be relevant to contractions involving joint rotation (i.e., dynamic). The purpose was to investigate the effect of calcaneal tendon stiffness on the dynamic RTD and rate of velocity development (RVD) in plantar flexor muscles. Young adult males (<i>n</i>&#xa0;=&#xa0;13) and females (<i>n</i>&#xa0;=&#xa0;2) performed prone isometric- and isotonic-mode maximal voluntary plantar flexion contractions (MVC). Ultrasound imaging was used to quantify tendon morphological characteristics to estimate Young's elastic modulus (YM). Maximal v

In [75]:
iteration_id = 4.222
text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
# text_df= pd.DataFrame({
#     'title': 'Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output',
#     'publication': 'Appl Physiol Nutr Metab',
#     'body': 'body'
# }, index=[0])
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3


array(['PURPOSE OF REVIEW: This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.\nRECENT FINDINGS: Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.\nSUMMARY: Sleep is an important modulator of neuroendocrine function and glucose metabolism and sleep loss has been shown to result in metabolic and endocrine alterations, including decreased glucose tolerance, decreased insulin

## 4.4

In [76]:
import requests
import pandas as pd
import json
import re
import os
import string
api_key = os.getenv('api_ncbi')
from db_orm import * 

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    # print(f'Data keys: {data.keys()}')
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    print(f'Number of abstract sections: {len(abstract_matches)}')
    if len(abstract_matches) > 1:
        cleaned_abstract_sections = []
        for match in abstract_matches:
            # clean_match = re.sub(r'<AbstractText(.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
            cleaned_abstract_sections.append(clean_match)
            
        abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
    else:
        abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''

    # abstract = 'abstract'

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'publication': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'publication': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df

iteration_id = 4.4
article_limit = 1

# text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
text_df= pd.DataFrame({
    'title': 'Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output',
    'publication': 'Appl Physiol Nutr Metab',
    'body': 'body'
}, index=[0])
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Search term: (Calcaneal tendon stiffness is  associated with dynamic time-dependent contractile output [ti]) AND (Appl Physiol Nutr Metab [ta])
Number of abstract sections: 1


array(["The ability to rapidly generate muscular torque and velocity is important in specialized activities and daily tasks of living. Tendon stiffness is one factor in the neuromuscular system that influences musculoskeletal torque transmission. Previous studies have reported weak-to-moderate correlations between tendon stiffness and rate of torque development (RTD). However, these correlations have been reported only for isometric contractions, which may not be relevant to contractions involving joint rotation (i.e., dynamic). The purpose was to investigate the effect of calcaneal tendon stiffness on the dynamic RTD and rate of velocity development (RVD) in plantar flexor muscles. Young adult males (<i>n</i>&#xa0;=&#xa0;13) and females (<i>n</i>&#xa0;=&#xa0;2) performed prone isometric- and isotonic-mode maximal voluntary plantar flexion contractions (MVC). Ultrasound imaging was used to quantify tendon morphological characteristics to estimate Young's elastic modulus (YM). Maximal v

In [77]:
iteration_id = 4.41
text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
# text_df= pd.DataFrame({
#     'title': 'Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output',
#     'publication': 'Appl Physiol Nutr Metab',
#     'body': 'body'
# }, index=[0])
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3


array(['PURPOSE OF REVIEW: This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.<br>RECENT FINDINGS: Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.<br>SUMMARY: Sleep is an important modulator of neuroendocrine function and glucose metabolism and sleep loss has been shown to result in metabolic and endocrine alterations, including decreased glucose tolerance, decreased ins

In [78]:
iteration_id = 4.42
article_limit = 2
text_df = get_table(table='gpt_queue', limit=article_limit, order='ASC') # db_orm.py
# text_df= pd.DataFrame({
#     'title': 'Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output',
#     'publication': 'Appl Physiol Nutr Metab',
#     'body': 'body'
# }, index=[0])
references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 2
Search term: Mediterranean diet adherence is associated with lower dementia risk, independent of genetic predisposition: findings from the UK Biobank prospective cohort study [ti]
Number of abstract sections: 4
Search term: Mediterranean diet adherence is associated with lower dementia risk, independent of genetic predisposition: findings from the UK Biobank prospective cohort study [ti]
Number of abstract sections: 4


array(['BACKGROUND: The identification of effective dementia prevention strategies is a major public health priority, due to the enormous and growing societal cost of this condition. Consumption of a Mediterranean diet (MedDiet) has been proposed to reduce dementia risk. However, current evidence is inconclusive and is typically derived from small cohorts with limited dementia cases. Additionally, few studies have explored the interaction between diet and genetic risk of dementia.<br>METHODS: We used Cox proportional hazard regression models to explore the associations between MedDiet adherence, defined using two different scores (Mediterranean Diet Adherence Screener [MEDAS] continuous and Mediterranean diet Pyramid [PYRAMID] scores), and incident all-cause dementia risk in 60,298 participants from UK Biobank, followed for an average 9.1&#xa0;years. The interaction between diet and polygenic risk for dementia was also tested.<br>RESULTS: Higher MedDiet adherence was associated with lo

In [84]:
references_df_dict[iteration_id]

,id,title,body,sent_to_sources,section,publication,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,85,"Mediterranean diet adherence is associated with lower dementia risk, independent of genetic predisposition: findings from the UK Biobank prospective cohort study",Background\nPreventing dementia is a global public health priority due to the enormous and growing societal cost of this condition [1]. A key strategy to reduce dementia incidence is the identific...,False,,None,"BACKGROUND: The identification of effective dementia prevention strategies is a major public health priority, due to the enormous and growing societal cost of this condition. Consumption of a Medi...",BMC medicine,"Oliver M Shannon, Janice M Ranson, Sarah Gregory, Helen Macpherson, Catherine Milte, Marleen Lentjes, Angela Mulligan, Claire McEvoy, Alex Griffiths, Jamie Matu, Tom R Hill, Ashley Adamson, Mario ...",2023,,21,1,81,,10.1186/s12916-023-02772-3,Background\nPreventing dementia is a global public health priority due to the enormous and growing societal cost of this condition [1]. A key strategy to reduce dementia incidence is the identific...
1,86,"Mediterranean diet adherence is associated with lower dementia risk, independent of genetic predisposition: findings from the UK Biobank prospective cohort study","Discussion\nUsing data from over 60,000 participants, we demonstrated that higher adherence to the MedDiet is associated with lower risk of incident all-cause dementia. Specifically, participants ...",False,discussion,None,"BACKGROUND: The identification of effective dementia prevention strategies is a major public health priority, due to the enormous and growing societal cost of this condition. Consumption of a Medi...",BMC medicine,"Oliver M Shannon, Janice M Ranson, Sarah Gregory, Helen Macpherson, Catherine Milte, Marleen Lentjes, Angela Mulligan, Claire McEvoy, Alex Griffiths, Jamie Matu, Tom R Hill, Ashley Adamson, Mario ...",2023,,21,1,81,,10.1186/s12916-023-02772-3,"Discussion\nUsing data from over 60,000 participants, we demonstrated that higher adherence to the MedDiet is associated with lower risk of incident all-cause dementia. Specifically, participants ..."


# *End of Page*